In [1]:
from google.colab import userdata
my_secret_key=userdata.get('HF_TOKEN')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install unsloth_zoo

In [4]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [5]:
# preparing quantized params
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # rank of weights
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.11.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
from datasets import load_dataset, concatenate_datasets
from unsloth.chat_templates import get_chat_template

# 1. Setup Tokenizer
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

# 2. LOAD AND PROCESS WIKI_QA (The Factual Data)
wiki_dataset = load_dataset("microsoft/wiki_qa", split="train")
# Filter for correct answers only (Label 1)
wiki_dataset = wiki_dataset.filter(lambda x: x['label'] == 1)

# Convert WikiQA to the "messages" format
def format_wiki_to_messages(example):
    return {
        "messages": [
            {"role": "user", "content": example["question"]},
            {"role": "assistant", "content": example["answer"]}
        ]
    }

# Apply conversion and remove old columns
wiki_dataset = wiki_dataset.map(format_wiki_to_messages)
wiki_dataset = wiki_dataset.select_columns(['messages'])

# 3. LOAD YOUR CUSTOM DATASET
custom_dataset = load_dataset("json", data_files="/content/concatenated_datasets.jsonl", split="train")
custom_dataset = custom_dataset.select_columns(["messages"])

# 4. MERGE THEM
combined_dataset = concatenate_datasets([wiki_dataset, custom_dataset])
combined_dataset = combined_dataset.shuffle(seed=42)

# 5. APPLY CHAT TEMPLATE
def apply_chat_template(examples):
    texts = []
    for conversation in examples['messages']:
        formatted_text = tokenizer.apply_chat_template(
            conversation,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(formatted_text)
    return {'text': texts}

combined_dataset = combined_dataset.map(apply_chat_template, batched=True)

# 6. SPLIT - create train, validation, and holdout test set
# First, separate out the holdout set (10% for evaluation team)
initial_split = combined_dataset.train_test_split(test_size=0.1, seed=42)
holdout_dataset = initial_split['test']

# Then split the remaining 90% into train and validation
train_val_split = initial_split['train'].train_test_split(test_size=0.1, seed=42)
train_dataset = train_val_split['train']
val_dataset = train_val_split['test']

# Save the holdout set for the evaluation team
holdout_dataset.to_json("holdout_evaluation_set.jsonl")

print(f"Training on {len(train_dataset)} examples")
print(f"Validation set: {len(val_dataset)} examples")
print(f"Holdout set (saved to file): {len(holdout_dataset)} examples")

README.md: 0.00B [00:00, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/594k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/264k [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/2.00M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/6165 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/14912 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Training on 12078 examples
Validation set: 1342 examples
Holdout set (saved to file): 1492 examples


In [7]:
train_dataset[0]["messages"] # checking conversations

[{'content': 'Hi!', 'role': 'user'},
 {'content': 'Hey!', 'role': 'assistant'},
 {'content': "how's week so far?", 'role': 'user'},
 {'content': 'pretty good, excited for thanksgiving you?',
  'role': 'assistant'},
 {'content': 'Same. Going to catch up on my reading! do you have a feast planned?',
  'role': 'user'},
 {'content': 'yup! gonna have dinner with family what books are you gonna read?',
  'role': 'assistant'},
 {'content': 'Something Deeply Hidden by Sean Carroll <REDACTED_LINK>',
  'role': 'user'},
 {'content': 'ahh nice what kind of books do you usually like to read',
  'role': 'assistant'},
 {'content': 'largely scifi How about you?', 'role': 'user'},
 {'content': 'yeah I also like scifi and some non-fiction too have you read Seveneves by Neal Stephenson?',
  'role': 'assistant'},
 {'content': "I haven't what is it about?", 'role': 'user'},
 {'content': "the moon is shattered by an asteroid and the pieces of the moon end up clouding the earth's atmostphere which causes alm

In [8]:
train_dataset[0]["text"] # trained sequence

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHi!<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHey!<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nhow's week so far?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\npretty good, excited for thanksgiving you?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nSame. Going to catch up on my reading! do you have a feast planned?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nyup! gonna have dinner with family what books are you gonna read?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nSomething Deeply Hidden by Sean Carroll <REDACTED_LINK><|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nahh nice what kind of books do you usually like to read<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nlargely scifi How about you?<|eot_id|><|start_header_i

In [9]:
from google.colab import drive
drive.mount('/content/drive')

# Create a permanent folder in your Drive
output_dir = "/content/drive/MyDrive/llama3.1-checkpoints"

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from transformers import EarlyStoppingCallback


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,# tokens per input
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        per_device_train_batch_size=2, # batch size
        gradient_accumulation_steps=16, # reduce VRAM Used
        warmup_ratio=0.1,
        num_train_epochs=1,
        learning_rate=2e-4, # learinig rate
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=50,
        optim="adamw_8bit", # gradient optimzier
        weight_decay=0.01,# regulization term
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="/content/drive/MyDrive/llama3.1-checkpoints", # Ensure this points to Drive
        report_to="none",  # Use WandB or other loggers if needed
        eval_strategy="steps",  # <-- Run validation periodically
        eval_steps=100,               # <-- Validate every steps
        save_strategy="steps",       # <-- Save checkpoint periodically
        save_steps=100,
        save_total_limit=3,
        load_best_model_at_end=True, # <-- Keep best model by validation loss
        metric_for_best_model="eval_loss",
        max_grad_norm=1.0,
    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # <-- Early stopping
)
checkpoint_path = "/content/drive/MyDrive/llama3.1-checkpoints/checkpoint-300"
trainer_stats = trainer.train(resume_from_checkpoint = checkpoint_path)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/12078 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1342 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,078 | Num Epochs = 1 | Total steps = 378
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 16 x 1) = 32
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss


OSError: [Errno 30] Read-only file system: '/content/drive/MyDrive/llama3.1-checkpoints/README.md'

In [ ]:
trainer_stats = trainer.train(resume_from_checkpoint = checkpoint_path)

In [11]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=6):   0%|          | 0/12078 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/1342 [00:00<?, ? examples/s]

In [ ]:
# trainer_stats = trainer.train()

##saving the checkpionts to google drive (IMPORTANT)

In [ ]:
# import shutil
# import os

# # 1. Make sure the destination folder exists in Google Drive
# # (Your 'output_dir' variable from the previous cell)
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# # 2. Copy the checkpoint folder from Colab to Google Drive
# # Source path (where it is now)
# source_path = "/content/outputs/checkpoint-300"

# # Destination path (where you want it in Drive)
# # We join paths to create: /content/drive/MyDrive/llama3.1-checkpoints/checkpoint-200
# destination_path = os.path.join(output_dir, "checkpoint-300")

# # Copy the directory
# print(f"Copying from {source_path} to {destination_path}...")
# shutil.copytree(source_path, destination_path)
# print("Success! Checkpoint saved to Google Drive.")

Copying from /content/outputs/checkpoint-300 to /content/drive/MyDrive/llama3.1-checkpoints/checkpoint-300...
Success! Checkpoint saved to Google Drive.


In [12]:
from transformers import TextStreamer
# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Set the chat template for inference
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

user_prompt = (
    "I feel like I am a bad person."
)

# Format the prompt for the model
messages = [{"role": "user", "content": user_prompt}]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# creating attention mask
attention_mask = (inputs != tokenizer.pad_token_id).long()

# Generate and stream the response
text_streamer = TextStreamer(tokenizer, skip_prompt=True,skip_special_tokens=True)
_ = model.generate(
    input_ids=inputs,
    streamer=text_streamer,
    attention_mask=attention_mask,
    max_new_tokens=2084,
    use_cache=True
)

Why do you think you are a bad person?


In [13]:
# Save the LoRA adapters on Colab. This is the FINAL MODEL
model.save_pretrained("llama3.1_finetuned")
tokenizer.save_pretrained("llama3.1_finetuned")

('llama3.1_finetuned/tokenizer_config.json',
 'llama3.1_finetuned/special_tokens_map.json',
 'llama3.1_finetuned/chat_template.jinja',
 'llama3.1_finetuned/tokenizer.json')

### Saving the model to HuggingFace

In [14]:
# Login to Hugging Face
from huggingface_hub import login
login(token=my_secret_key)

# Push the model
model.push_to_hub("pierreramez/llama3.1-finetuned-v2", tokenizer=tokenizer)
tokenizer.push_to_hub("pierreramez/llama3.1-finetuned-v2")

README.md:   0%|          | 0.00/626 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          |  557kB /  168MB            

Saved model to https://huggingface.co/pierreramez/llama3.1-finetuned-v2


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpz2kzpkip/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

## some functions to continue chatting with the model

In [15]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model)

# Conversation history
messages = []

def chat(user_input):
    # Add user message to history
    messages.append({"role": "user", "content": user_input})

    # Tokenize the full conversation
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    attention_mask = (inputs != tokenizer.pad_token_id).long()

    # Generate response
    output = model.generate(
        input_ids=inputs,
        attention_mask=attention_mask,
        max_new_tokens=512,
        use_cache=True,
        pad_token_id=tokenizer.pad_token_id,
    )

    # Decode only the new tokens
    response = tokenizer.decode(output[0][inputs.shape[1]:], skip_special_tokens=True)

    # Add assistant response to history
    messages.append({"role": "assistant", "content": response})

    return response

# To reset conversation, run: messages = []